# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'inference/get-started',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-Coder-480B-A35B-Instruct',
 '/Qwen/Qwen3-235B-A22B-Instruct-2507',
 '/bosonai/higgs-audio-v2-generation-3B-base',
 '/tencent/HunyuanWorld-1',
 '/zai-org/GLM-4.5',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/smola/higgs_audio_v2',
 '/spaces/zumjoy/Multi-Style_Video-to-Anime_Generator',
 '/spaces/Qwen/Qwen3-Coder-WebDev',
 '/spaces/akhaliq/anycoder',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/interstellarninja/hermes_reasoning_tool_use',
 '/datasets/MegaScience/MegaScience',
 '/datasets/microsoft/rStar-Coder',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',


In [12]:
get_links_user_prompt(huggingface)

'Here is the list of links on the website of https://huggingface.co - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\nLinks (some might be relative links):\n/\n/models\n/datasets\n/spaces\n/docs\n/enterprise\n/pricing\n/login\n/join\ninference/get-started\n/spaces\n/models\n/Qwen/Qwen3-Coder-480B-A35B-Instruct\n/Qwen/Qwen3-235B-A22B-Instruct-2507\n/bosonai/higgs-audio-v2-generation-3B-base\n/tencent/HunyuanWorld-1\n/zai-org/GLM-4.5\n/models\n/spaces/enzostvs/deepsite\n/spaces/smola/higgs_audio_v2\n/spaces/zumjoy/Multi-Style_Video-to-Anime_Generator\n/spaces/Qwen/Qwen3-Coder-WebDev\n/spaces/akhaliq/anycoder\n/spaces\n/datasets/fka/awesome-chatgpt-prompts\n/datasets/NousResearch/Hermes-3-Dataset\n/datasets/interstellarninja/hermes_reasoning_tool_use\n/datasets/MegaScience/MegaScience\n/datasets/microsoft/rStar-Coder\n/datasets\n/join\n/pricing#endp

In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/#about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on

In [26]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short but very attractive, creative and noticeable brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-Coder-480B-A35B-Instruct\nUpdated\n5 days ago\n•\n10.1k\n•\n844\nbosonai/higgs-audio-v2-generation-3B-base\nUpdated\nabout 8 hours ago\n•\n67k\n•\n399\ntencent/HunyuanWorld-1\nUpdated\nabout 14 hours ago\n•\n380\nQwen/Qwen3-235B-A22B-Instruct-2507\nUpdated\n8 d

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is an innovative platform leading the AI community in building the future. We provide a collaborative environment for machine learning enthusiasts, researchers, and developers to create, share, and innovate around machine learning models, datasets, and applications.

## Our Mission
Our mission is to democratize AI, making it accessible for everyone. By building an ecosystem around open-source tools and fostering collaboration, Hugging Face aims to empower individuals and organizations in developing cutting-edge AI solutions.

## Our Offerings
### Models
With over **1 million** machine learning models available, Hugging Face serves as a comprehensive hub for model discovery and collaboration. Whether you're looking for state-of-the-art NLP models or innovative computer vision solutions, you'll find it here.

### Datasets
Explore our extensive collections of **250,000+ datasets** tailored for various ML tasks. We facilitate seamless access and sharing to support research and development efforts across the AI community.

### Spaces
Hugging Face provides a platform for running AI applications easily. From generating web applications to transforming videos into unique styles, our Spaces allow users to deploy and showcase their AI creations effortlessly.

### Enterprise Solutions
For organizations, Hugging Face offers **enterprise-grade services** that include advanced security, dedicated support, and scalable compute resources starting at **$20/user/month**. Join over **50,000 organizations** globally leveraging our infrastructure, including notable names like Google, Microsoft, and Amazon.

## Community Engagement
Our collaborative culture is at the heart of everything we do. We believe in open-source and community-driven development. Join our forums, contribute to our model and dataset repositories, and be a part of the evolving AI landscape.

## Why Choose Hugging Face?
- **Cutting-edge Tools**: Access state-of-the-art models and innovative frameworks like Transformers and Diffusers.
- **Vibrant Community**: Engage with a passionate community that shares knowledge and supports each other's growth through collaborative projects.
- **Career Opportunities**: We are always on the lookout for talents who are eager to make an impact in AI. Check out our [careers page](#) to explore job openings and join our team.

## Join Us
Embark on your journey in machine learning with Hugging Face today!

- **Website**: [huggingface.co](https://huggingface.co)
- **Follow Us**: [Twitter](#), [LinkedIn](#), [GitHub](#)

Hugging Face—The AI community building the future!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face
**The AI community building the future.** Hugging Face is a collaborative platform designed to empower machine learning enthusiasts and professionals to come together and build state-of-the-art AI models, datasets, and applications.

### Our Mission
At Hugging Face, we aim to create a community-driven approach to artificial intelligence, making it accessible and collaborative for everyone. With robust tools and a dynamic ecosystem, we support developers and organizations in leveraging AI to solve complex challenges.

## Key Offerings
- **Models**: Explore a vast library of over **1 million models**. From text generation to computer vision, we have something for everyone.
- **Datasets**: Access and share **250,000+ datasets** for various machine learning tasks, paving the way for innovation and excellence.
- **Spaces**: Collaborate and deploy applications easily across our platforms, with more than **400,000 applications** already thriving in our ecosystem.
- **Enterprise Solutions**: Tailored offerings for businesses, including advanced security, support, and resource management.

## Customers
Join a diverse pool of organizations leveraging Hugging Face for their AI needs. Over **50,000 organizations** trust us to power their machine learning projects, including major players like:
- Microsoft
- Google
- Amazon
- Grammarly
- Meta

## Company Culture
Hugging Face prides itself on a culture of collaboration and open-source openness. We encourage creativity, foster innovation, and support continuous learning. Our community-driven approach means that every member has a voice, contributing to a vibrant ecosystem of AI developers and researchers.

### Work Environment
- **Inclusive Community**: We believe in diversity and inclusion, ensuring everyone feels a sense of belonging.
- **Continuous Learning**: We support ongoing education and skill development, providing resources and opportunities for professional growth.
- **Collaborative Projects**: Engage in exciting projects with our community where your work can have a real impact.

## Careers at Hugging Face
We are always on the lookout for passionate individuals who want to make a difference in the AI space. By joining Hugging Face, you’ll become part of a dynamic team that is dedicated to building the future of AI. Open positions cover various roles in engineering, data science, and community management.

### Why Work with Us?
- Be part of a pioneering company at the forefront of AI technology.
- Contribute to meaningful projects that impact lives.
- Work in a flexible and innovative environment that respects work-life balance.

### Join Us!
**Ready to take your career to the next level?** Explore available positions on our [Careers page](https://huggingface.co/jobs) and become part of our community!

---

Hugging Face is not just a platform; it’s a movement towards democratizing AI for everyone. With a commitment to quality and community, we invite you to explore, collaborate, and innovate with us. Let’s build the future together!

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}



# Welcome to Hugging Face: Where AI Meets Warm Fuzzies! 🤗

## Who Are We?
At **Hugging Face**, we’re not just building the future—we’re doing it with a smile! We’re a vibrant community of AI enthusiasts, developers, and model makers, working together to create cutting-edge AI tools and applications. We hug—sorry, **collaborate**—over models, datasets, and a whole lot more!

## What Do We Offer?
1. **1 Million+ Models**: That's right, Netflix may have 1 million shows, but we have 1 million models! Whether you want to generate text, images, or even 3D art, we’ve got you covered! 

2. **Dazzling Datasets**: With over 250k datasets, you’ll never run out of data to analyze. Who knew datasets could be so... exciting? 🤓 

3. **Spaces**: Our virtual playground where you can run your AI applications and test out funky new integrations like DeepSite v2 and Multi-Style Video-to-Anime Generator! Because who doesn't want to live in an anime world? 🎉

## Meet Our Customers
From tiny startups to towering behemoths like **Amazon, Google,** and **Microsoft**, we’re honored to have over **50,000 organizations** using Hugging Face! We cater to everyone—whether you're a corporate giant or a playful AI hobbyist. We even have a nonprofit section, just to keep things charitable (and to make your heart a little fluffier). 

## Our Company Culture 🥳
At Hugging Face, we believe in collaboration, creativity, and making AI accessible for everyone. Our team works like a well-oiled machine (or robot, if you will). We promote an open-source atmosphere where **sharing is caring**! No secret handshake; just good vibes and endless opportunities to learn from each other!

### Perks of Working Here:
- **Passionate People**: Our team is a mix of AI wizards, language ninjas, and creativity aficionados. We promise you won't be bored!
- **Flexibility**: Work remotely but also feel the love of a tight-knit community. It's like your favorite sweater—but for work!
- **Career Growth**: Wherever you are in your career, we help you grow like a well-watered plant. 🌱
  
## Join Us! 📩
Interested in being part of our AI adventure? We want you! Check out our job listings and bring your dream to life. Remember, working at Hugging Face might just lead to a superpower: making AI better and more fun every day!

## Wrapping It Up
Let's make AI stunningly simple, incredibly powerful, and a whole lot of fun! Whether you’re here to build, learn, or experiment—let’s cuddle up with some AI magic! 

**Ready to Hug It Out?** 🤗
[Visit Hugging Face Now!](https://huggingface.co) 

**P.S.** Our mascot is a fluffy face—seriously, get ready for a warm welcome!



In [27]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# 🌟 Welcome to Hugging Face! 🌟

## The AI Community Building the Future

At **Hugging Face**, we are reshaping the landscape of artificial intelligence and machine learning. With a vibrant community comprising over **50,000 organizations** worldwide—from startups to tech giants like Meta, Amazon, and Google—our platform provides a collaborative environment where innovators can explore and share **over 1 million models** and **250,000 datasets**. 

### 🚀 What We Offer:

- **Models & Datasets**: Explore and contribute to a massive hub of state-of-the-art models and comprehensive datasets tailored for every ML task.
- **Spaces**: Run your own applications and collaborate on limitless projects, enabling you to create and innovate without boundaries. 
- **Enterprise Solutions**: Join leading organizations in elevating your AI capabilities with secure, scalable, and specialized tools designed to meet the needs of your business.

### 🤝 Our Culture:

At Hugging Face, we believe **collaboration is key**. Our culture thrives on:
- **Open Source**: Pioneering free and open-source tools that empower developers to build here, there, and everywhere.
- **Community Focus**: Engaging with passionate developers, researchers, and creators to foster innovation at every level. 
- **Inclusivity**: Welcoming diverse talents and perspectives that enrich our environment and drive our mission forward.

### 🌌 Join Us:

**Careers at Hugging Face**:  
We're always on the lookout for talented individuals who are eager to make an impact in the AI landscape. Whether you're a seasoned expert or a passionate newcomer, you’ll find a home here where your contributions are valued, and growth is prioritized. Check our [Jobs page](https://huggingface.co/jobs) for current openings!

### 💬 Connect With Us:

Join the conversation and explore our community on:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface/)
- [Discord](https://discord.gg/huggingface)

### 📈 Ready to Build the Future?

Dive into the world of AI at **[Hugging Face](https://huggingface.co)** and help us make machine learning more accessible, efficient, and impactful for everyone. 

---

**Together, let's create the future of AI!**



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>